#### GOAL: 
Create a model which predicts a Role given a Job Description
    
##### Input Example: 
'Randstad Technical di Vimercate seleziona un manutentore per inserimento in somministrazione presso azienda del settore gomma plastica. La risorsa si occuperà della manutenzione ordinaria e straordinaria di presse a iniezione idrauliche  ed elettriche, di attrezzature elettromeccaniche e degli impianti del plant produttivo. Requisiti: diploma o qualifica in ambito meccanico o elettromeccanico; esperienza nella mansione di almeno 3 anni; disponibilità al lavoro sia a giornata che su tre turni; flessibilità, precisione, capacità organizzative, proattività'

##### Model outputs:
'Elettromeccanici'

### Import dependencies

In [26]:
# !pip install -r requirements.txt
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
import numpy as np
from IPython.display import Image
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giorgiomondauto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giorgiomondauto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/giorgiomondauto/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


### Load the data

In [2]:
professioni = pd.read_csv('data/professioni.csv')
dati_aprile = pd.read_csv('Job_vacancies_aprile_completo.csv', skiprows=1, usecols = [1,2,3],\
                          names = ['Target','Sub_Role','Job_Description'])
dati_aprile = dati_aprile.reindex(np.random.permutation(dati_aprile.index))
print('dati_aprile shape {}'.format(dati_aprile.shape))
dati_aprile[['Target','Job_Description']].head() # per facilitare la comprensione dei dati di testo da parte del modello

dati_aprile shape (650, 3)


,Target,Job_Description
107,Cuochi in alberghi e ristoranti,"La risorsa, inserita nel dinamico team di cuci..."
268,Statistici,I candidati ideali hanno conseguito la laurea ...
323,Tecnici web,Web master Esperienza minima: 3 anni Tipo di c...
385,Tecnici web,"Programmatore Web Ravenna, Descrizione Selezio..."
29,Elettromeccanici,La risorsa verra' inserita in un contesto molt...


In [3]:
dati_aprile.Target.value_counts()

Commessi delle vendite al minuto    149
Statistici                          104
Camerieri di ristorante             100
Cuochi in alberghi e ristoranti     100
Elettromeccanici                     99
Tecnici web                          98
Name: Target, dtype: int64

### Text Preprocessing on Professioni csv

In [4]:
# professioni.Subgroup = professioni.Subgroup.apply(lambda x: x.lower().split('\n'))
# professioni.subgroup1 = professioni.subgroup1.apply(lambda x: x.lower().split('\n'))
# professioni.subgroup2 = professioni.subgroup2.apply(lambda x: x.lower().split('\n'))
# professioni.Subgroup = professioni.Subgroup + professioni.subgroup1 + professioni.subgroup2
# professioni.Subgroup = professioni.Subgroup.apply(lambda x: list(set(x)))

In [5]:
# professioni.head()

### Create a dictionary for professioni. e.g. 
{Statistici : ['attuario', 'data scientist', 'esperto in ricerca operativa',..... ]}

In [6]:
# professioni_dictionary = pd.Series(professioni.Subgroup.values,index=professioni.Group).to_dict()
# # to remove empty space from the subgroup list of values
# for i in range(0,professioni.Subgroup.shape[0]):
#     while("" in professioni.Subgroup.iloc[i]) : 
#         professioni.Subgroup.iloc[i].remove("") 

In [7]:
# professioni_dictionary

In [8]:
# # I need this step to convert back
# subgroup_dict = {}
# for group, subgroups in professioni_dictionary.items():
#     for subgroup in subgroups:
#         subgroup_dict[subgroup] = group
# subgroup_dict

### Text Preprocessing on Job Descripion column

#### Tokenization
split the text into words

In [11]:
# remove punctuation
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
dati_aprile.Target = dati_aprile.Target.apply(lambda x: x.lower()) #tutte le parole in minuscolo 
dati_aprile.Job_Description = dati_aprile['Job_Description'].apply(lambda x: word_tokenize(x.lower())) # lowercase each word
dati_aprile[['Target','Job_Description']].head()

,Target,Job_Description
107,cuochi in alberghi e ristoranti,"[la, risorsa, inserita, nel, dinamico, team, d..."
268,statistici,"[i, candidati, ideali, hanno, conseguito, la, ..."
323,tecnici web,"[web, master, esperienza, minima, anni, tipo, ..."
385,tecnici web,"[programmatore, web, ravenna, descrizione, sel..."
29,elettromeccanici,"[la, risorsa, verra, inserita, in, un, contest..."


remove words with lenght < 3 (e.g. numbers and not meaningful words)

In [12]:
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if len(i) >= 3])
dati_aprile.Job_Description.head() # note the word 'vimercate' in row = 0

107    [risorsa, inserita, nel, dinamico, team, cucin...
268    [candidati, ideali, hanno, conseguito, laurea,...
323    [web, master, esperienza, minima, anni, tipo, ...
385    [programmatore, web, ravenna, descrizione, sel...
29     [risorsa, verra, inserita, contesto, molto, am...
Name: Job_Description, dtype: object

In [13]:
##### remove in 'job_description' words from 'fonte'
Fonti = ['randstad', 'monster', 'infojob', 'technical', 'kelly', 'services', 'italia', 'lavoropi',\
             'quanta','adeccp','spa','vimercate','temporary','openjobmetis','agenzia']
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in Fonti])
dati_aprile.Job_Description.head(5) # compare row 0 and row 2 with previous row 0 and row 2

107    [risorsa, inserita, nel, dinamico, team, cucin...
268    [candidati, ideali, hanno, conseguito, laurea,...
323    [web, master, esperienza, minima, anni, tipo, ...
385    [programmatore, web, ravenna, descrizione, sel...
29     [risorsa, verra, inserita, contesto, molto, am...
Name: Job_Description, dtype: object

#### Stopwords
Stopwords are words generally not relevant to a text; thereby we get rid of them

In [17]:
list_stopwords = list(set(stopwords.words('italian')))
print('Example of stopwords:',list_stopwords[:20])
print(10 * '%')
print('lenth of stopwords: {}'.format(len(list_stopwords)))
# molte sono state tagliate nel passaggio precedente when i len < 4

Example of stopwords: ['sarebbero', 'vostri', 'fareste', 'faceva', 'dallo', 'chi', 'stette', 'era', 'farebbero', 'agl', 'tua', 'questo', 'ebbero', 'sarà', 'su', 'voi', 'ai', 'abbiate', 'da', 'sarai']
%%%%%%%%%%
lenth of stopwords: 279


In [18]:
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in list_stopwords] )
dati_aprile.Job_Description.head(5)

107    [risorsa, inserita, dinamico, team, cucina, oc...
268    [candidati, ideali, conseguito, laurea, specia...
323    [web, master, esperienza, minima, anni, tipo, ...
385    [programmatore, web, ravenna, descrizione, sel...
29     [risorsa, verra, inserita, contesto, molto, am...
Name: Job_Description, dtype: object

In [19]:
# wordcloud of main words
# def cloud(data,backgroundcolor = 'white', width = 800, height = 600):
#     wordcloud = WordCloud(stopwords = STOPWORDS, background_color = backgroundcolor,
#                          width = width, height = height).generate(data)
#     plt.figure(figsize = (15, 10))
#     plt.imshow(wordcloud)
#     plt.axis("off")
#     plt.show()

In [20]:
# job_description = dati_aprile['Job_Description'].apply(lambda x: ' '.join(x))
# cloud(' '.join(job_description)) # most common words

#### Remove words referring to: Comuni, Provincie, Regioni

In [21]:
# load a text file containing a list of 'Comuni, Provincie, Regioni' italiane
geo_data = pd.read_fwf('data/listacomuni.txt')
geo_data.to_csv('data/listacomuni.csv')
geo_data = pd.read_csv('data/listacomuni.csv', sep=';')
comune = [i.lower() for i in geo_data.Comune]
geo_data.Provincia = geo_data.Provincia.apply(lambda x: str(x))
provincia = [i.lower() for i in geo_data.Provincia]
regione = [i.lower() for i in geo_data.Regione] + ['lazio']
# print('Comuni samples:',comune[:10])
# print(20*'%')
# print('Provincia samples:',provincia[:10])
# print(20*'%')
# print('Regione samples:',regione[:10])

In [22]:
def remove_comuni(data, column, geo_data):
    ''' function to remove geographic noisy'''
    comune = [i.lower() for i in geo_data.Comune]
    geo_data.Provincia = geo_data.Provincia.apply(lambda x: str(x))
    provincia = [i.lower() for i in geo_data.Provincia]
    regione = [i.lower() for i in geo_data.Regione]  + ['lazio']
    
    data[column] = data[column].apply(lambda x: [i for i in x if i not in comune])
    data[column] = data[column].apply(lambda x: [i for i in x if i not in provincia])
    data[column] = data[column].apply(lambda x: [i for i in x if i not in regione])
    
    return data

In [23]:
dati_aprile[['Target','Job_Description']].head()

,Target,Job_Description
107,cuochi in alberghi e ristoranti,"[risorsa, inserita, dinamico, team, cucina, oc..."
268,statistici,"[candidati, ideali, conseguito, laurea, specia..."
323,tecnici web,"[web, master, esperienza, minima, anni, tipo, ..."
385,tecnici web,"[programmatore, web, ravenna, descrizione, sel..."
29,elettromeccanici,"[risorsa, verra, inserita, contesto, molto, am..."


In [24]:
dati_aprile = remove_comuni(dati_aprile, 'Job_Description', geo_data)
dati_aprile[['Target','Job_Description']].head()

,Target,Job_Description
107,cuochi in alberghi e ristoranti,"[risorsa, inserita, dinamico, team, cucina, oc..."
268,statistici,"[candidati, ideali, conseguito, laurea, specia..."
323,tecnici web,"[web, master, esperienza, minima, anni, tipo, ..."
385,tecnici web,"[programmatore, web, descrizione, selezioniamo..."
29,elettromeccanici,"[risorsa, verra, inserita, contesto, molto, am..."


##### Get rid of unusual_words : misspelled words - uncommon words¶

In [27]:
italian_vocab = pd.read_fwf('data/660000_parole_italiane.txt', names = ['word'])
italian_vocab = set(italian_vocab.word.tolist())
english_vocab = sorted(set(w.lower() for w in nltk.corpus.words.words())) # english vocabulary

In [28]:
def uncommon_words(data,column, italian_vocab, english_vocab = None):
    data[column] = data[column].apply(lambda x: [i for i in x if i in italian_vocab])
    return data

In [29]:
dati_aprile = uncommon_words(dati_aprile, 'Job_Description', italian_vocab, english_vocab)
dati_aprile[['Target','Job_Description']].head()# note that the word 'vimercate' is not present anymore

,Target,Job_Description
107,cuochi in alberghi e ristoranti,"[risorsa, inserita, dinamico, team, cucina, pr..."
268,statistici,"[candidati, ideali, conseguito, laurea, specia..."
323,tecnici web,"[web, master, esperienza, minima, anni, tipo, ..."
385,tecnici web,"[programmatore, web, descrizione, selezioniamo..."
29,elettromeccanici,"[risorsa, verra, inserita, contesto, molto, am..."


In [30]:
stemmer = nltk.stem.snowball.ItalianStemmer(ignore_stopwords=True)

def stemming(dataset):
    ''' stemming a text '''
    stemmer = nltk.stem.snowball.ItalianStemmer(ignore_stopwords=False)
    dataset['Job_Description'] = dataset['Job_Description'].apply(lambda x: [stemmer.stem(i) \
                                                                        for i in x])
    return dataset

dati_aprile = stemming(dati_aprile)

In [31]:
dati_aprile.head()

,Target,Sub_Role,Job_Description
107,cuochi in alberghi e ristoranti,cuoco-grigliere/cuoca-grigliera,"[risors, inser, dinam, team, cucin, principal,..."
268,statistici,DATA SCIENTISTS / MACHINE LEARNING ENGINEERS,"[candid, ideal, consegu, laure, specialist, di..."
323,tecnici web,Web master,"[web, master, esperit, minim, anni, tip, contr..."
385,tecnici web,programmatore web,"[programm, web, descrizion, selezion, azi, cli..."
29,elettromeccanici,Avvolgitore motori elettrici,"[risors, verr, inser, contest, molt, ampi, str..."


In [32]:
noisy_words = ['degli','tipi','del','dei','delle','per','nel','dell',\
               'sui','less','sul','che','jsss','teor','dal','della']
dati_aprile['Job_Description'] = dati_aprile['Job_Description'].apply(lambda x: [i for i in x if i not in noisy_words])
dati_aprile['Job_Description'] .head()

107    [risors, inser, dinam, team, cucin, principal,...
268    [candid, ideal, consegu, laure, specialist, di...
323    [web, master, esperit, minim, anni, tip, contr...
385    [programm, web, descrizion, selezion, azi, cli...
29     [risors, verr, inser, contest, molt, ampi, str...
Name: Job_Description, dtype: object

In [33]:
# %matplotlib inline
# matplotlib.style.use('ggplot')
# field = "Job_Description"
# Role = list(dati_aprile.Target.unique())
# for i in Role:
#     txt = dati_aprile[dati_aprile['Target']==i][field]
#     txt = [i for x in txt.tolist() for i in x]
#     words_except_stop_dist = nltk.FreqDist(w for w in txt) 
#     words_except_stop_dist
#     rslt = pd.DataFrame(words_except_stop_dist.most_common(35),
#                             columns=[i.upper(), 'Frequency']).set_index(i.upper())
#     print(rslt)
#     matplotlib.style.use('ggplot')
#     rslt.plot.bar(rot=0)
#     print(100 *'%')

In [34]:
# %matplotlib inline
# matplotlib.style.use('ggplot')
# field = "Job_Description"
# Role = list(dati_aprile.Target.unique())
# wordcount = []
# for i in Role:
#     txt = dati_aprile[dati_aprile['Target']==i][field]
#     txt = [i for x in txt.tolist() for i in x]
#     words_except_stop_dist = nltk.FreqDist(w for w in txt) 
    
#     wordcount.append(pd.DataFrame(words_except_stop_dist.most_common(10),
#                             columns=[i.upper(), 'Frequency']).set_index(i.upper()))

In [35]:
# dati = pd.concat(wordcount,axis = 0)
# dati['parola'] = dati.index
# ruolo = []
# for i in Role:
#     ruolo.append([i] * 10)
# dati['ruolo'] = [i for x in ruolo for i in x]

In [36]:
# dati.to_csv('../dati_tableau_text.csv')

##### Create Role dictionary and encode the column Role

In [37]:
Role_dictionary = pd.Series(dati_aprile['Target'].unique()).to_dict()
Role_dictionary = dict([(value, key) for key, value in Role_dictionary.items()])
Role_dictionary

{'cuochi in alberghi e ristoranti': 0,
 'statistici': 1,
 'tecnici web': 2,
 'elettromeccanici': 3,
 'commessi delle vendite al minuto': 4,
 'camerieri di ristorante': 5}

In [ ]:
dati_aprile['Multi_Class'] = dati_aprile['Target'].apply(lambda x: Role_dictionary.get(x,'Unknown'))
dati_aprile[['Target','Multi_Class','Job_Description']]

#### Have a look at bigrams

In [89]:
# from nltk import bigrams, collocations
# Text = dati_aprile.Job_Description.iloc[1]
# bigrams_from_text = bigrams(Text)
# list(bigrams_from_text)[:6]

In [90]:
# bigrams_from_text = bigrams(Text)

# list(iter(dict(nltk.FreqDist(i for i in list(bigrams_from_text))).items()))[:6]

In [33]:
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: ' '.join(x))
dati_aprile.Job_Description.head()

539    addett banc salum risors verrann inser contrat...
341    webmaster azi titol stud diplom matur settor e...
504    risors vend assistt client manten banc pul ord...
169    lavor filial ricerc struttur sanitar aiut cuoc...
602    addett banc pescher principal mansion occup ri...
Name: Job_Description, dtype: object

### Data Split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(dati_aprile['Job_Description'], 
                                                    dati_aprile['Multi_Class'],
                                                   test_size=0.20, random_state=42)

print('Number of rows in the total set: {}'.format(dati_aprile.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 650
Number of rows in the training set: 520
Number of rows in the test set: 130


### Bag of Words processing to our dataset

In [44]:
# # list of text documents
# text = ['Io ho esperienza come cuoco in ristoranti e alberghi. Vorrei candidarmi come cuoco o aiuto cuoco']
# print(text)
# print(10*'#')
# vectorizer = CountVectorizer()
# # tokenize and build vocab
# vectorizer.fit(text)
# vectorizer.vocabulary_['solo'] = max(vectorizer.vocabulary_.values()) + 1
# print(vectorizer.vocabulary_)

# # encode document
# vector = vectorizer.transform(text)
# # summarize encoded vector
# print(10*'#')
# print('Le parole uniche sono: ',vector.shape[1])
# print(10*'#')
# print(vector.toarray())

In [45]:
# from sklearn.feature_extraction.text import CountVectorizer
# count_vector = CountVectorizer()

# count_vector.fit(X_train)
# # # Fit the training data and then return the matrix
# training_data = count_vector.transform(X_train)
# with open("count_vector.pkl", 'wb') as handle:
#                     pickle.dump(count_vector, handle)
        
# with open("Role_dictionary.pkl", 'wb') as handle:
#                 pickle.dump(Role_dictionary, handle)       

# # Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
# testing_data = count_vector.transform(X_test)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
count_vector.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [47]:
with open('lista_skills.pkl', 'rb') as handle:
    lista = pickle.load(handle)

In [48]:
noisy_words = ['degli','tipi','del','dei','delle','per','nel','dell',\
               'sui','less','sul','che','jsss','teor','dal','della']
lista_skills = [word_tokenize(i) for i in list(lista)]
lista_skills = [i for x in lista_skills for i in x if i not in noisy_words and  len(i) >2]
lista_skills = [stemmer.stem(i) for i in lista_skills if i not in count_vector.vocabulary_.keys()]
lista_skills 

['attiv',
 'vend',
 'scienz',
 'vinicol',
 'assicur',
 'settor',
 'rivest',
 'mural',
 'rivest',
 'pav',
 'specif',
 'softw',
 'ingegner',
 'elettr',
 'specif',
 'uva',
 'smalltalk',
 'programm',
 'informat',
 'norm',
 'prodott',
 'origin',
 'animal',
 'istruzion',
 'fabbric',
 'material',
 'audiovis',
 'anatom',
 'pesc',
 'teor',
 'insiem',
 'stamp',
 'scritt',
 'apl',
 'common',
 'lisp',
 'event',
 'sport',
 'norm',
 'mater',
 'apparecchiatur',
 'elettr',
 'biotecnolog',
 'psicolog',
 'cognit',
 'floricoltur',
 'attiv',
 'ricreat',
 'softw',
 'uffic',
 'metod',
 'analis',
 'prestazion',
 'tic',
 'ajax',
 'process',
 'birrif',
 'utensil',
 'meccan',
 'caratterist',
 'serviz',
 'vin',
 'frizzant',
 'strateg',
 'commercializz',
 'conten',
 'process',
 'controll',
 'cred',
 'indagin',
 'opinion',
 'attiv',
 'promozional',
 'reazion',
 'allerg',
 'cosmet',
 'adob',
 'illustrator',
 'ingegner',
 'meccan',
 'elettr',
 'prolog',
 'programm',
 'informat',
 'nutrient',
 'dolcium',
 'acust',
 '

In [49]:
# to create a new dictionary starting from a 'x' value
new_dict = dict((v,k) for k,v in enumerate(set(lista_skills),max(count_vector.vocabulary_.values()) + 1 )) # instead of max(  ) + 1 we can type any number
count_vector.vocabulary_.update(new_dict)

In [50]:
# Fit the training data and then return the matrix
training_data = count_vector.transform(X_train)
with open("count_vector2104.pkl", 'wb') as handle:
                    pickle.dump(count_vector, handle)
        
with open("Role_dictionary.pkl", 'wb') as handle:
                pickle.dump(Role_dictionary, handle)       

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

In [51]:
# from scipy import sparse
# sparse.save_npz("training_data.npz", training_data)
# sparse.save_npz("testing_data.npz", testing_data)

# training_data = sparse.load_npz("training_data.npz")
# testing_data = sparse.load_npz("testing_data.npz")

In [52]:
print('training data',training_data.shape)
print('testing data',testing_data.shape)

training data (520, 2582)
testing data (130, 2582)


### Naive Bayes implementation

In [53]:
# # y_train.to_csv('y_train.csv')
# import pandas as pd
# y_train = pd.read_csv('y_train.csv',usecols= ['Multi_Class'])
# y_train = y_train.Multi_Class
# y_train

In [54]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)
import joblib
joblib.dump(naive_bayes, 'model2104.sav')

['model2104.sav']

In [55]:
# Image(filename ="Naive.png", width=1000, height=300)

## P(Statistico|    Words = [dati, matematica, analisi, …...., inglese] )

In [56]:
def get_key(value, dictionary):
    for k,v in dictionary.items():
        if value == v:
            return k

In [57]:
predictions = naive_bayes.predict(testing_data)
predictions_keys = []
for i in predictions:
    predictions_keys.append(get_key(i,Role_dictionary))
print('Prediction for rows {} are \n {}'.format(X_test.index.tolist()[:10],predictions_keys[:10]))

print(100*'%')
actual_predictions = []
for i in y_test.tolist():
    actual_predictions.append(get_key(i,Role_dictionary))
print('actual predictions are: \n {}'.format(actual_predictions[:10]))

Prediction for rows [436, 429, 154, 234, 482, 115, 530, 130, 199, 549] are 
 ['camerieri di ristorante', 'camerieri di ristorante', 'cuochi in alberghi e ristoranti', 'statistici', 'camerieri di ristorante', 'cuochi in alberghi e ristoranti', 'commessi delle vendite al minuto', 'cuochi in alberghi e ristoranti', 'statistici', 'commessi delle vendite al minuto']
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
actual predictions are: 
 ['camerieri di ristorante', 'camerieri di ristorante', 'cuochi in alberghi e ristoranti', 'statistici', 'camerieri di ristorante', 'cuochi in alberghi e ristoranti', 'commessi delle vendite al minuto', 'cuochi in alberghi e ristoranti', 'statistici', 'commessi delle vendite al minuto']


In [58]:
# check the accuracy of our model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test,predictions)))

Accuracy score:  0.9230769230769231
